In [89]:

# coding: utf-8

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from time import localtime, strftime
from pyspark.sql import SQLContext

import urllib
import json
import requests
import pandas as pd

# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext(appName="CitiBike_download")
sqlContext = SQLContext(sc)

fileNameList = ['station_information','station_status','system_alerts','system_regions']  
cleanedData = []
strData = ''

# function to clean the Json
def jSonCleaner(strData):
    strData = r.text[r.text.find("["):-2]
    return strData


for nfile in fileNameList:
    #removing unwanted string from the starting of the json file and from starting    
    #reading the json file using requests
    r = requests.get("https://gbfs.citibikenyc.com/gbfs/en/" +nfile + ".json")
    # correct json after cleaning
    cleanedData.append(jSonCleaner(r.text)) 


# ##  Getting JSON  List from formatted string
station_information = json.loads(cleanedData[0])
station_status = json.loads(cleanedData[1])
system_alerts = json.loads(cleanedData[2])
system_regions = json.loads(cleanedData[3])

#Changing Python List to Pandas DF
pdf_station_info= pd.DataFrame(station_information)
pdf_station_status= pd.DataFrame(station_status)
pdf_system_alerts= pd.DataFrame(system_alerts)
pdf_system_regions= pd.DataFrame(system_regions)


# Datatype change : station_ids -> List to station_id string and renaming the column too
columns = ['alert_id','last_updated', 'station_id','summary','type']
new_pdf_system_alerts = pd.DataFrame(columns=columns)
for index, row in pdf_system_alerts.iterrows():
    if len(row.station_ids)==1:
        new_pdf_system_alerts.loc[len(new_pdf_system_alerts)]=[row['alert_id'],row['last_updated'],row['station_ids'][0],row['summary'],row['type']] 
    else:
        for val in row.station_ids[:]:
            new_pdf_system_alerts.loc[len(new_pdf_system_alerts)]=[row['alert_id'],row['last_updated'],val,row['summary'],row['type']]

#Changing Pandas DF to SQL for aggregation 
sqldf_station_info=sqlContext.createDataFrame(pdf_station_info)   
sqldf_station_status=sqlContext.createDataFrame(pdf_station_status)
if len(pdf_system_alerts>1):
    sqldf_system_alerts=sqlContext.createDataFrame(new_pdf_system_alerts)  
if len(pdf_system_regions):
    sqldf_system_regions=sqlContext.createDataFrame(pdf_system_regions)




joined_info_status = sqldf_station_info.join(sqldf_station_status,['station_id'],'outer')
join_info_status_alert = joined_info_status.join(sqldf_system_alerts,['station_id'],'outer')
join_info_sta_alt_reg = join_info_status_alert.join(sqldf_system_regions,['region_id'],'outer')


pd_df=join_info_sta_alt_reg.toPandas()
pd_df['Date']=strftime("%Y/%m/%d %H:%M:%S", localtime())


from time import gmtime, strftime
timestr = strftime("%Y%m%d_%H%M%S", localtime())
pd_df.to_csv('/citibike/output/'+timestr+'.csv',index=False)

sc.stop()



In [90]:
import pandas as pd

pdf_station_info= pd.DataFrame(station_information)
pdf_station_status= pd.DataFrame(station_status)
pdf_system_alerts= pd.DataFrame(system_alerts)
pdf_system_regions= pd.DataFrame(system_regions)

In [91]:
pdf_system_alerts

,alert_id,last_updated,station_ids,summary,type
0,6,1512130928,[402],Valet service available,OTHER
1,8,1512131463,[304],Valet service available,OTHER
2,46,1512131181,[426],Valet service available,OTHER
3,49,1512140375,[3443],Valet full,OTHER
4,58,1512132474,[281],Valet service available,OTHER
5,79,1512131847,[484],Valet service available,OTHER


In [84]:
#Changing Pandas DF to SQL for aggregation 

sqldf_station_info=sqlContext.createDataFrame(pdf_station_info)   
sqldf_station_status=sqlContext.createDataFrame(pdf_station_status)
if len(new_pdf_system_alerts>1):
    print(" --------------In new_pdf_system_alerts ----------------")
    sqldf_system_alerts=sqlContext.createDataFrame(new_pdf_system_alerts)  
if len(pdf_system_regions):
    sqldf_system_regions=sqlContext.createDataFrame(pdf_system_regions)

print(type(sqldf_system_regions))
print(type(join_info_sta_alt_reg))
joined_info_status = sqldf_station_info.join(sqldf_station_status,['station_id'],'outer')
join_info_status_alert = joined_info_status.join(sqldf_system_alerts,['station_id'],'outer')
join_info_sta_alt_reg = join_info_status_alert.join(sqldf_system_regions,['region_id'],'outer')


join_info_status_alert.head(1)


<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


[Row(station_id=u'296', capacity=35, eightd_has_key_dispenser=False, lat=40.71413089, lon=-73.9970468, name=u'Division St & Bowery', region_id=71.0, rental_methods=[u'CREDITCARD', u'KEY'], rental_url=u'http://app.citibikenyc.com/S6Lr/IBV092JufD?station_id=296', short_name=u'5238.04', eightd_has_available_keys=False, is_installed=1, is_renting=1, is_returning=1, last_reported=1512153751, num_bikes_available=14, num_bikes_disabled=1, num_docks_available=20, num_docks_disabled=0, alert_id=None, last_updated=None, summary=None, type=None)]

In [85]:
from time import localtime, strftime


joined_info_status.createOrReplaceTempView('aggregation')
result = sqlContext.sql('select * from aggregation')


pd_df=joined_info_status.toPandas()
pd_df['Date']=strftime("%Y/%m/%d %H:%M:%S", localtime())


from time import gmtime, strftime
timestr = strftime("%Y%m%d_%H%M%S", localtime())
pd_df.to_csv('/citibike/output/'+timestr+'.csv',index=False)

sc.stop()


In [88]:
sc.stop()